## Steps performed on case study
- Loading data set
- Data cleaning
- Imputing null values
- Filtering high value customers
- Deriving target variable
- New predictor variable derivation
- EDA
- Data preparation
- Logistic Regression
- PCA
- Decision tree
- Random forest
- XGBoost
- Feature importance and business conclusions

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)

In [2]:
# import required libraries
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve ,confusion_matrix , precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

## Loading the data set

In [3]:
# read data
churn = pd.read_csv("telecom_churn_data.csv")

In [4]:
churn.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,offnet_mou_7,offnet_mou_8,offnet_mou_9,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_ic_mou_9,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,roam_og_mou_9,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2t_mou_9,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2m_mou_9,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2f_mou_9,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_t2c_mou_9,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,loc_og_mou_9,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2t_mou_9,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2m_mou_9,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2f_mou_9,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_t2c_mou_9,std_og_mou_6,std_og_mou_7,std_og_mou_8,std_og_mou_9,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,isd_og_mou_9,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,spl_og_mou_9,og_others_6,og_others_7,og_others_8,og_others_9,total_og_mou_6,total_og_mou_7,total_og_mou_8,total_og_mou_9,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2t_mou_9,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2m_mou_9,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_t2f_mou_9,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,loc_ic_mou_9,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2t_mou_9,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2m_mou_9,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2f_mou_9,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_t2o_mou_9,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,std_ic_mou_9,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,total_ic_mou_9,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,spl_ic_mou_9,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,isd_ic_mou_9,ic_others_6,ic_others_7,ic_others_8,ic_others_9,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_num_9,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,total_rech_amt_9,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,max_rech_amt_9,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,last_day_rch_amt_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9,total_rech_data_6,total_rech_data_7,total_rech_data_8,total_rech_data_9,max_rech_data_6,max_rech_data_7,max_rech_data_8,max_rech_data_9,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_2g_9,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,count_rech_3g_9,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,av_rech_amt_data_9,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_2g_mb_9,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,vol_3g_mb_9,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_3g_9,arpu_2g_6,arpu_2g_7,arpu_2g_8,arpu_2g_9,night_pck_user_6,night_pck_user_7,night_pck_user_8,night_pck_user_9,monthly_2g_6,monthly_2g_7,monthly_2g_8,monthly_2g_9,sachet_2g_6,sachet_2g_7,sachet_2g_8,sachet_2g_9,monthly_3g_6,monthly_3g_7,monthly_3g_8,monthly_3g_9,sachet_3g_6,sachet_3g_7,sachet_3g_8,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,214.816,213.803,21.100,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,0.00,0.00,0.00,0.00,NaN,NaN,0.16,NaN,NaN,NaN,4.13,NaN,NaN,NaN,1.15,NaN,NaN,NaN,5.44,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00,0.00,5.44,0.00,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,4,3,

In [5]:
churn.shape

(99999, 226)

In [6]:
churn.info(verbose = 1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 226 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   mobile_number             int64  
 1   circle_id                 int64  
 2   loc_og_t2o_mou            float64
 3   std_og_t2o_mou            float64
 4   loc_ic_t2o_mou            float64
 5   last_date_of_month_6      object 
 6   last_date_of_month_7      object 
 7   last_date_of_month_8      object 
 8   last_date_of_month_9      object 
 9   arpu_6                    float64
 10  arpu_7                    float64
 11  arpu_8                    float64
 12  arpu_9                    float64
 13  onnet_mou_6               float64
 14  onnet_mou_7               float64
 15  onnet_mou_8               float64
 16  onnet_mou_9               float64
 17  offnet_mou_6              float64
 18  offnet_mou_7              float64
 19  offnet_mou_8              float64
 20  offnet_mou_9              f

As observed, there are 99999 rows and 226 columns in the dataset.

In [ ]:
churn.describe(include='all')

In [ ]:
#Differentiating columns based upon type
id_cols = ['mobile_number', 'circle_id']

date_cols = ['last_date_of_month_6',
             'last_date_of_month_7',
             'last_date_of_month_8',
             'last_date_of_month_9',
             'date_of_last_rech_6',
             'date_of_last_rech_7',
             'date_of_last_rech_8',
             'date_of_last_rech_9',
             'date_of_last_rech_data_6',
             'date_of_last_rech_data_7',
             'date_of_last_rech_data_8',
             'date_of_last_rech_data_9'
            ]

cat_cols =  ['night_pck_user_6',
             'night_pck_user_7',
             'night_pck_user_8',
             'night_pck_user_9',
             'fb_user_6',
             'fb_user_7',
             'fb_user_8',
             'fb_user_9'
            ]

num_cols = [column for column in churn.columns if column not in id_cols + date_cols + cat_cols]

print('Number of ID columns: {0}'.format(len(id_cols)))
print('Number of Date columns: {0}'.format(len(date_cols)))
print('Number of Category columns: {0}'.format(len(cat_cols)))
print('Number of Numercical columns: {0}'.format(len(num_cols)))
print('Total columns in dataset: {0}'.format(churn.shape[1]))

## Missing values inspection

In [ ]:
#Percentage of missing values column-wise
round(churn.isnull().sum()*100/churn.shape[0], 2)

## Imputing null values

In [ ]:
#Recharge related columns
recharge_cols = ['total_rech_num_6', 'total_rech_num_7', 'total_rech_num_8', 'total_rech_num_9',
                 'total_rech_amt_6', 'total_rech_amt_7', 'total_rech_amt_8', 'total_rech_amt_9',
                 'max_rech_amt_6', 'max_rech_amt_7', 'max_rech_amt_8', 'max_rech_amt_9',
                 'date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_9',
                 'last_day_rch_amt_6', 'last_day_rch_amt_7', 'last_day_rch_amt_8', 'last_day_rch_amt_9',
                 ]

churn[recharge_cols].describe()

In [ ]:
churn[recharge_cols].isnull().sum()

Getting the recharge amount for which recharge date is not present

In [ ]:
churn[recharge_cols].loc[churn[recharge_cols].date_of_last_rech_6.isnull(), 
                         ['total_rech_num_6', 'date_of_last_rech_6']].count()

In [ ]:
churn[recharge_cols].loc[churn[recharge_cols].date_of_last_rech_7.isnull(), 
                         ['total_rech_num_7', 'date_of_last_rech_7']].count()

In [ ]:
churn[recharge_cols].loc[churn[recharge_cols].date_of_last_rech_8.isnull(), 
                         ['total_rech_num_8', 'date_of_last_rech_8']].count()

In [ ]:
churn[recharge_cols].loc[churn[recharge_cols].date_of_last_rech_9.isnull(), 
                         ['total_rech_num_9', 'date_of_last_rech_9']].count()

#### As observed, recharge value is 0 for when the recharge date is not present. So this is a meaningful missing, we leave it as it is since we will drop the recharge dates later

## Analysing data recharge features

In [ ]:
#We observed that the recharge data columns have around 74% null values, but we need this feature to later 
#filter out high value customers. Hence, replacing the null values with 0.0

churn['total_rech_data_6'] = churn['total_rech_data_6'].replace(np.NaN,0.0)

churn['total_rech_data_7'] = churn['total_rech_data_7'].replace(np.NaN,0.0)

churn['total_rech_data_8'] = churn['total_rech_data_8'].replace(np.NaN,0.0)

churn['av_rech_amt_data_6'] = churn['av_rech_amt_data_6'].replace(np.NaN,0.0)

churn['av_rech_amt_data_7'] = churn['av_rech_amt_data_7'].replace(np.NaN,0.0)

churn['av_rech_amt_data_8'] = churn['av_rech_amt_data_8'].replace(np.NaN,0.0)

In [ ]:
#adding new column total recharge amount for data: total_rech_amt_data for filtering High Value customer later

churn['total_rech_amt_data_6'] = churn.av_rech_amt_data_6 * churn.total_rech_data_6

churn['total_rech_amt_data_7'] = churn.av_rech_amt_data_7 * churn.total_rech_data_7

churn['total_rech_amt_data_8'] = churn.av_rech_amt_data_8 * churn.total_rech_data_8

In [ ]:
#We can drop the avg recharge amount and total recharge amount columns
cols_to_drop = ['total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8',
                'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8']
churn.drop(cols_to_drop, axis = 1, inplace = True)

### Replacing NaN in categorical columns

In [ ]:
# replacing Nan in categorical columns with new category '-1'
churn[cat_cols] = churn[cat_cols].apply(lambda x: x.fillna('-1'))

In [ ]:
round(churn[cat_cols].isnull().sum() * 100 / churn.shape[0], 2)

### Dropping columns with high percentage of missing

In [ ]:
#dropping columns with more than 50% missing values
MISSING_THRESHOLD = 0.5
include_cols = list(churn.apply(lambda column: True if column.isnull().sum()/churn.shape[0] < MISSING_THRESHOLD else False))
churn = churn.loc[:, include_cols]
churn.shape

In [ ]:
include_cols.count(False)

### Dropping ID and Date columns

In [ ]:
date_cols = ['last_date_of_month_6',
             'last_date_of_month_7',
             'last_date_of_month_8',
             'last_date_of_month_9',
             'date_of_last_rech_6',
             'date_of_last_rech_7',
             'date_of_last_rech_8',
             'date_of_last_rech_9'
            ]
churn = churn.drop(date_cols, axis=1)
print("Shape after dropping: ", churn.shape)

### Imputing using IterativeImputer

In [ ]:
dropped_cols = ['max_rech_data_9', 'count_rech_2g_6', 'max_rech_data_6', 'av_rech_amt_data_7', 'count_rech_3g_7', 
                'arpu_3g_9', 'total_rech_data_8', 'arpu_2g_9', 'arpu_2g_8', 'count_rech_3g_8', 'count_rech_3g_9', 
                'total_rech_data_6', 'arpu_3g_8', 'max_rech_data_8', 'arpu_3g_6', 'count_rech_2g_8', 'arpu_2g_6', 
                'max_rech_data_7', 'total_rech_data_7', 'arpu_2g_7', 'total_rech_data_9', 'count_rech_2g_7', 
                'av_rech_amt_data_9', 'av_rech_amt_data_6', 'arpu_3g_7', 'count_rech_2g_9', 'av_rech_amt_data_8', 
                'count_rech_3g_6']
num_cols = [x for x in num_cols if x not in dropped_cols]
churn[num_cols].head()

In [ ]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

In [ ]:
churn_cols = churn.columns
imputer = IterativeImputer(max_iter = 2, random_state = 10)
df_stg = churn.copy()
df_stg.drop(id_cols, axis=1, inplace=True)
churn_imputed = imputer.fit_transform(df_stg)

In [ ]:
# convert imputed numpy array to pandas dataframe
churn_filtered = pd.DataFrame(churn_imputed, columns=churn_cols.drop(id_cols))
print(churn_filtered.isnull().sum()*100/churn_filtered.shape[0])

## High-value customers

In the Indian and the southeast Asian market, approximately 80% of revenue comes from the top 20% customers (called high-value customers). Thus, if we can reduce churn of the high-value customers, we will be able to reduce significant revenue leakage.
Those who have recharged with an amount more than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months (the good phase).

In [ ]:
churn_filtered.info(verbose=1)

In [ ]:
churn_filtered.head()

In [ ]:
# calculate the total data recharge amount for June and July --> number of recharges * average recharge amount
churn_filtered['avg_rech_amt_6_7'] = (churn_filtered.total_rech_amt_6 + churn_filtered.total_rech_amt_data_6
                                    + churn_filtered.total_rech_amt_7 + churn_filtered.total_rech_amt_data_7) / 2
churn_filtered.head()

In [ ]:
# look at the 70th percentile recharge amount
print("Recharge amount at 70th percentile: {0}".format(churn_filtered.avg_rech_amt_6_7.quantile(0.7)))

In [ ]:
# retain only those customers who have recharged their mobiles with more than or equal to 70th percentile amount
churn_filtered = churn_filtered.loc[churn_filtered.avg_rech_amt_6_7 > churn_filtered.avg_rech_amt_6_7.quantile(0.7), :]
churn_filtered = churn_filtered.reset_index(drop=True)
churn_filtered.shape

In [ ]:
churn_filtered = churn_filtered.drop('avg_rech_amt_6_7', axis = 1)
churn_filtered.shape

#### As observed, after filteration, we are left with 29953 rows and 187 columns

## Deriving churn value

The customers who do not have any call recharge and data recharge are considered to have churned

In [ ]:
#Calculate total incoming and outgoing minutes of usage
churn_filtered['total_calls_mou_9'] = churn_filtered.total_ic_mou_9 + churn_filtered.total_og_mou_9

In [ ]:
#Calculate 2g and 3g data consumption
churn_filtered['total_internet_mb_9'] =  churn_filtered.vol_2g_mb_9 + churn_filtered.vol_3g_mb_9

In [ ]:
# create churn variable: those who have not used either calls or internet in the month of September are customers who have churned
# 0 - not churn, 1 - churn
churn_filtered['churn'] = churn_filtered.apply(lambda x: 1 if (x.total_calls_mou_9 == 0 and x.total_internet_mb_9 == 0) else 0, axis=1)
churn_filtered.head()

In [ ]:
# delete derived variables
churn_filtered = churn_filtered.drop(['total_calls_mou_9', 'total_internet_mb_9'], axis=1)

In [ ]:
# print churn ratio
print('Churn class')
print(churn_filtered.churn.value_counts()*100/churn_filtered.shape[0])

#### As observed, there is high class imbalance in churning. There is a 92:8 ratio

## Delete columns that are related to 9th month

In [ ]:
# delete all features relating to 9th month
churn_filtered = churn_filtered.filter(regex='[^9]$', axis=1)
churn_filtered.shape

### Updating the categorical and numerical columns

In [ ]:
# extract all names that end with 9
col_9_names = churn.filter(regex='9$', axis=1).columns
print(col_9_names)

In [ ]:
# update num_cols and cat_cols column name list
cat_cols = [col for col in cat_cols if col not in col_9_names]
num_cols = [col for col in churn_filtered.columns if col not in cat_cols]

print('Categorical columns: ' , len(cat_cols))
print(cat_cols)
print()
print('Numerical columns: ' , len(num_cols))
print(num_cols)

In [ ]:
# change columns types
churn_filtered[num_cols] = churn_filtered[num_cols].apply(pd.to_numeric)
churn_filtered[cat_cols] = churn_filtered[cat_cols].apply(lambda column: column.astype("category"), axis=0)

In [ ]:
churn_filtered.info(verbose = 1)

## New variables derivation

Since 8th month is the deciding month for the customer whether he will churn or not, let's try to find some differences between recharge and daat usage by the customer. We will calculate difference in behavior for all different types of column

In [ ]:
churn_filtered['arpu_diff'] = churn_filtered.arpu_8 - ((churn_filtered.arpu_6 + churn_filtered.arpu_7)/2)

churn_filtered['onnet_mou_diff'] = churn_filtered.onnet_mou_8 - ((churn_filtered.onnet_mou_6 + churn_filtered.onnet_mou_7)/2)

churn_filtered['monthly_3g_diff'] = churn_filtered.monthly_3g_8 - ((churn_filtered.monthly_3g_6 + churn_filtered.monthly_3g_7)/2)

churn_filtered['roam_ic_mou_diff'] = churn_filtered.roam_ic_mou_8 - ((churn_filtered.roam_ic_mou_6 + churn_filtered.roam_ic_mou_7)/2)

churn_filtered['roam_og_mou_diff'] = churn_filtered.roam_og_mou_8 - ((churn_filtered.roam_og_mou_6 + churn_filtered.roam_og_mou_7)/2)

In [ ]:
churn_filtered.shape

## Remove columns with no variance

In [ ]:
for col in churn_filtered.columns:
    if churn_filtered[col].nunique() == 1:
        print("Column", col ,"has no variance and contains only ", churn_filtered[col].nunique()," unique value")
        print("Dropping the column", col)
        print()
        churn_filtered.drop(col, axis = 1, inplace = True)

print("Shape of the updated dataset:", churn_filtered.shape)

## Exploratory data analysis

In [ ]:
# plot the churn ratio
plt.figure(figsize = [5,5])
plt.title('Churn', fontsize = 15)
churn_filtered.churn.value_counts(normalize=True).plot.pie(autopct='%1.1f%%', labels=['Churned', 'Not churned'])
plt.show()

#### As observed, there is a class imbalance in the target variable, i.e., churn

In [ ]:
sns.distplot(churn_filtered.arpu_diff)

#### We observe a normal plot for average revenue per user difference (8th - avg(6th and 7th)) with mean zero

In [ ]:
plt.figure(figsize = [15,8])
plt.subplot(1,2,1)
plt.title('Monthly 3g difference - Churned')
sns.distplot(churn_filtered.loc[churn_filtered.churn == 1].monthly_3g_diff, kde = True)
plt.subplot(1,2,2)
plt.title('Monthly 3g difference - Not churned')
sns.distplot(churn_filtered.loc[churn_filtered.churn == 0].monthly_3g_diff, kde = True)

#### For churned users we observe that values <=0 have higher frequency density, where as in non-churn the frequency density is higher on the positive side

In [ ]:
tenure_data = churn_filtered.copy()
plt.figure(figsize=(14,8))
# aon --> Age on network - number of days the customer is using the operator's network
tenure_data['tenure'] = tenure_data['aon'] / 30
sns.distplot(tenure_data['tenure'], hist = True, kde = False,
             bins = int(180/5), color = 'blue', 
             hist_kws = {'edgecolor':'red'},
             kde_kws = {'linewidth': 4})
plt.ylabel('Number of Customers')
plt.xlabel('Tenure in Months')
plt.title('Customers Vs Tenure')
plt.show()

#### As observed, customers tend to leave the operator service after sometime. The customers are not loyal to the service, maybe because of the schemes or service or network.

In [ ]:
#function to observer behavior difference in 6th, 7th and 8th month
def plot_box_chart(attribute):
    plt.figure(figsize=(20,16))
    df = churn_filtered
    plt.subplot(2,3,1)
    plt.title('Month 6', fontsize = 15)
    sns.boxplot(data = df, y = attribute + "_6", x = "churn", hue = "churn",
                showfliers = False, palette= ("plasma"))
    plt.subplot(2,3,2)
    plt.title('Month 7', fontsize = 15)
    sns.boxplot(data = df, y = attribute + "_7", x = "churn", hue = "churn",
                showfliers = False, palette = ("plasma"))
    plt.subplot(2,3,3)
    plt.title('Month 8', fontsize = 15)
    sns.boxplot(data = df, y = attribute + "_8", x = "churn", hue = "churn",
                showfliers = False, palette = ("plasma"))
    plt.show()

In [ ]:
# Recharge amount vs churn
plot_box_chart('max_rech_amt')

#### As observed, the value for max_rech_amt has dropped drastically in the 8th month for customers who have churned. For customers who have not churned, the max_rech_amt remains somewhat constant

In [ ]:
plot_box_chart('total_rech_amt_data')

#### As observed, there is a huge drop in total recharge amount for data in the 8th month (action phase) for churned customers.

In [ ]:
churn_filtered.columns

In [ ]:
sns.jointplot(x = churn_filtered.roam_ic_mou_diff, y = churn_filtered.roam_og_mou_diff)

In [ ]:
sns.boxplot(x = 'churn', y = 'arpu_diff', data = churn_filtered, showfliers = False)

In [ ]:
plt.figure(figsize = [8, 8])
plt.title('Number recharge vs Data recharge in month 8')
sns.scatterplot(x = 'max_rech_amt_8', y = 'total_rech_amt_data_8', hue = 'churn', data = churn_filtered)
plt.show()

#### As observed, there are some high-value customers who had high max_rech_amt in months na dstill churned. One possible reason for it can be a better scheme from competetive services. Also, all the churned customers have low total_rech_amt_data

In [ ]:
plt.figure(figsize = [8, 8])
plt.title('Arpu difference vs onnet_mou difference (8th - AVG(6th + 7th))')
sns.scatterplot(x = 'arpu_diff', y = 'onnet_mou_diff', hue = 'churn', data = churn_filtered)
plt.show()

In [ ]:
### Correlation matrix
# figure size
plt.figure(figsize=(15,15))
corr_matrix = churn_filtered.corr().abs()
# heatmap
sns.heatmap(corr_matrix, cmap="YlGnBu", annot=True)
plt.show()

## Remove highly correlated columns

In [ ]:
#Selecting columns which have absolute correlation greater than 60%
# Create correlation matrix
corr_matrix = churn_filtered.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]

# Drop features 
churn_filtered.drop(to_drop, axis = 1, inplace = True)

In [ ]:
churn_filtered.shape

## Updating the category and numerical columns

In [ ]:
cat_cols = churn_filtered.select_dtypes('category').columns
num_cols = [col for col in churn_filtered.columns if col not in cat_cols]

### dropping highly skewed columns

In [ ]:
skewnessThreshold = 80
def RemoveSkewedColumn(col, type):
    if col == 'churn':
        return
    count_dict = (round(churn_filtered[col].value_counts(normalize = True) * 100, 2)).to_dict()
    IsSkewed = list(map(lambda x: x >= 80, count_dict.values()))
    if IsSkewed.count(True) > 0:
        print(col, "Skewness:")
        #print(count_dict)
        churn_filtered.drop(col, axis = 1, inplace = True)
        print('Due to high skewness, this column was dropped')
        print()
    else:
        if type == 'category':
            #change type of column to category
            churn_filtered[col] = churn_filtered[col].astype('category')

In [ ]:
for col in num_cols:
    RemoveSkewedColumn(col, 'numeric')

In [ ]:
cat_cols = churn_filtered.select_dtypes('category').columns
num_cols = [col for col in churn_filtered.columns if col not in cat_cols]

In [ ]:
churn_filtered.shape

In [ ]:
churn_filtered.head()

### redraw the correlation matrix

In [ ]:

### Correlation matrix
# figure size
plt.figure(figsize=(15,15))
corr_matrix = churn_filtered.corr().abs()
# heatmap
sns.heatmap(corr_matrix, cmap="YlGnBu", annot=True)
plt.show()

## Treating outliers using IQR

Using IQR methodology to cap outliers

In [ ]:
for col in num_cols:
    Q1 = churn_filtered[col].quantile(0.25)
    Q3 = churn_filtered[col].quantile(0.75)
    IQR = Q3 - Q1
    if(len(churn_filtered[num_cols[0]][churn_filtered[num_cols[0]] < (Q1 - 1.5 * IQR)]) > 0 
       or len(churn_filtered[num_cols[0]][churn_filtered[num_cols[0]] > (Q3 + 1.5 * IQR)]) > 0):
        print('Found outliers in column: ' + col)
        churn_filtered[num_cols[0]][churn_filtered[num_cols[0]] < (Q1 - 1.5 * IQR)] = Q1 - 1.5 * IQR
        churn_filtered[num_cols[0]][churn_filtered[num_cols[0]] > (Q3 + 1.5 * IQR)] = Q3 + 1.5 * IQR
        print('Capped outliers in column "' + col + '" using IQR method' )

# Modelling

In [ ]:
result_column_names = ["Model", "Train Accuracy", "Test Accuracy", "Train Recall", "Test Recall", "Test F1 Score"]

model_results = pd.DataFrame(columns = result_column_names)

### Dummification of categorical columns

In [ ]:
churn_filtered.shape

In [ ]:
# Creating a dummy variable for some of the categorical variables and dropping the first one.
dummyVariables = pd.get_dummies(churn_filtered[cat_cols], drop_first=True)

In [ ]:
# Adding the results to the master dataframe
churn_filtered = pd.concat([churn_filtered, dummyVariables], axis=1)

In [ ]:
churn_filtered.drop(cat_cols, axis=1, inplace=True)

In [ ]:
churn_filtered.shape

### Train-test split

Splitting the data set in 7:3 ratio for train and test

In [ ]:
#Splitting the dataframe to 70% train and 30% test
df_train, df_test = train_test_split(churn_filtered, train_size=0.7, random_state=100)
print(df_train.shape)
print(df_test.shape)

In [ ]:
y_train = df_train['churn']
X_train = df_train.drop('churn', axis = 1)

In [ ]:
y_test = df_test['churn']
X_test = df_test.drop('churn', axis = 1)

## Scaling

Using a minmax scaler to scale data in train and test set

In [ ]:
# scaling 
scaler = MinMaxScaler()
num_cols.remove('churn')
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])
X_train.head()

## Logistic Regression Model using RFE

In [ ]:
# using sklearn 
logreg = LogisticRegression(class_weight='balanced')

In [ ]:
X_train.describe()

In [ ]:
rfe = RFE(logreg, 15)             # running RFE with 15 variables as output
rfe = rfe.fit(X_train, y_train)

In [ ]:
## list columns that rfe predicted as useful
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
# get supported columns from RFE
col = X_train.columns[rfe.support_]
col

### Assessing the model with StatsModels

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train, X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
### drop vol_3g_mb_diff since this has p value greater than expected
col = col.drop('loc_og_t2f_mou_6')
col

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Check VIf values again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
### drop total_rech_amt_data_7 since this has p value greater than expected
col = col.drop('monthly_3g_diff', 1)

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Check VIf values again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# drop loc_og_t2m_mou_diff as it has high p value
col = col.drop('loc_ic_t2f_mou_6')

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Check VIf values again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# drop loc_og_t2m_mou_diff as it has high p value
col = col.drop('roam_og_mou_diff')

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Check VIf values again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# drop loc_og_t2m_mou_diff as it has high p value
col = col.drop('spl_og_mou_8')

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Check VIf values again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# drop loc_og_t2m_mou_diff as it has high p value
col = col.drop('arpu_diff')

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Check VIf values again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# drop loc_og_t2m_mou_diff as it has high p value
col = col.drop('onnet_mou_diff')

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Check VIf values again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# drop loc_og_t2m_mou_diff as it has high p value
col = col.drop('loc_ic_t2t_mou_6')

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Check VIf values again
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
finalCols = X_train[col].columns
X_test_log = X_test[finalCols]
X_test_log.head()

In [ ]:
# using sklearn
logregfinal = LogisticRegression(class_weight='balanced')

In [ ]:
logregfinal.fit(X_train[finalCols], y_train)
y_test_pred = logregfinal.predict(X_test_log)
y_test_pred

#### Evaluation of model on train data

In [ ]:
y_train_lr_pred = logregfinal.predict(X_train[finalCols])

In [ ]:
# create onfusion matrix
y_train_lr_pred_final = pd.DataFrame({'Converted': y_train.values, 'Converted_Prob': y_train_lr_pred})
y_train_lr_pred_final['LeadNumber'] = y_train.index

y_train_lr_pred_final['predicted'] = y_train_lr_pred_final.Converted_Prob.map(lambda x: 1 if x > 0.35 else 0)
y_train_lr_pred_final.head()

# Let's check the overall accuracy.
rfe_log_train_acc = round(metrics.accuracy_score(y_train_lr_pred_final.Converted, y_train_lr_pred_final.predicted), 2)
rfe_log_train_acc

In [ ]:
# Let's see the sensitivity/recall of our logistic regression model
rfe_log_train_recall = recall_score(y_train_lr_pred_final.Converted, y_train_lr_pred_final.predicted)
rfe_log_train_recall

#### Evaluation of model on test data

In [ ]:
# create onfusion matrix
y_pred_final = pd.DataFrame({'Converted':y_test.values, 'Converted_Prob':y_test_pred})
y_pred_final['LeadNumber'] = y_test.index


y_pred_final['predicted'] = y_pred_final.Converted_Prob.map(lambda x: 1 if x > 0.35 else 0)
y_pred_final.head()

# Let's check the overall accuracy.
rfe_log_test_acc = round(metrics.accuracy_score(y_pred_final.Converted, y_pred_final.predicted), 2)
rfe_log_test_acc

#### We observe 71% accuracy on train data and 72% accuracy on test data at 0.35 probability cut off. Hence, we move forward for calculating the sensitivity of the model

In [ ]:
confusion_log = metrics.confusion_matrix(y_pred_final.Converted, y_pred_final.predicted )

In [ ]:
TP = confusion_log[1,1] # true positive 
TN = confusion_log[0,0] # true negatives
FP = confusion_log[0,1] # false positives
FN = confusion_log[1,0] # false negatives

In [ ]:
# Let's see the sensitivity/recall of our logistic regression model
rfe_log_test_recall = recall_score(y_pred_final.Converted, y_pred_final.predicted)
rfe_log_test_recall

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
#precision
precision_score(y_pred_final.Converted, y_pred_final.predicted)

In [ ]:
# f1 score
rfe_test_f1 = f1_score(y_pred_final.Converted, y_pred_final.predicted)
rfe_test_f1

In [ ]:
# result_column_names = ["Model", "Train Accuracy", "Test Accuracy", "Train Recall", "Test Recall", "Test F1 Score"]
rfe_temp_results = {
        result_column_names[0]:'Logistic Regression with RFE', 
        result_column_names[1]:rfe_log_train_acc,
        result_column_names[2]:rfe_log_test_acc,
        result_column_names[3]:rfe_log_train_recall,
        result_column_names[4]:rfe_log_test_recall,
        result_column_names[5]:rfe_test_f1,
}
model_results = model_results.append(rfe_temp_results, ignore_index=True)

## PCA

In [ ]:
pca = PCA(random_state = 42)
pca.fit(X_train)

In [ ]:
pd.Series(np.round(pca.explained_variance_ratio_.cumsum(), 4)*100)

In [ ]:
# plot feature variance
features = range(pca.n_components_)
cumulative_variance = np.round(np.cumsum(pca.explained_variance_ratio_)*100, decimals=4)
plt.figure(figsize=(175/20,100/20)) # 100 elements on y-axis; 175 elements on x-axis; 20 is normalising factor
plt.plot(cumulative_variance)

Looking at the explained variance ratio for each component

In [ ]:
pca.explained_variance_ratio_

#### Making a scree plot for the explained variance

In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)
var_cumu

In [ ]:
fig = plt.figure(figsize=[12,8])
plt.vlines(x=6, ymax=2, ymin=0, colors="r", linestyles="--")
plt.hlines(y=0.95, xmax=30, xmin=0, colors="g", linestyles="--")
plt.plot(var_cumu)
plt.ylabel("Cumulative variance explained")
plt.show()

In [ ]:
pca_final = PCA(0.99, svd_solver='full')

In [ ]:
df_train_pca = pca_final.fit_transform(X_train)

In [ ]:
df_train_pca.shape

In [ ]:
corrmat = np.corrcoef(df_train_pca.transpose())

In [ ]:
corrmat.shape

Plotting the heatmap of the corr matrix

In [ ]:
plt.figure(figsize=[15,15])
sns.heatmap(corrmat, annot=True)

Applying the transformation on the test set

In [ ]:
df_test_pca = pca_final.transform(X_test)
df_test_pca.shape

Looking at the explained variance ratio for each component

## Hyperparameter tuning - PCA and Logistic Regression

In [ ]:
# logistic regression - the class weight is used to handle class imbalance - it adjusts the cost function
logistic = LogisticRegression(class_weight='balanced')

# hyperparameter space
params = {'C': [0.08, 0.09, 0.1, 0.5, 1, 2, 3, 4, 5, 10], 'penalty': ['l1', 'l2']}

# create 5 folds
folds = KFold(n_splits = 5, shuffle = True, random_state = 4)

# create gridsearch object
model = GridSearchCV(estimator=logistic, cv=folds, param_grid=params, scoring='recall', n_jobs=-1, verbose=1)

In [ ]:
# fit model
model.fit(df_train_pca, y_train)

In [ ]:
# cross validation results
pd.DataFrame(model.cv_results_)

In [ ]:
# print best hyperparameters
print("Best AUC: ", model.best_score_)
print("Best hyperparameters: ", model.best_params_)
print(model.best_estimator_)

#### Evaluation on train data

In [ ]:
# predict churn on train data
y_pred_train = model.predict(df_train_pca)

In [ ]:
# check area under curve
y_pred_prob_train = model.predict_proba(df_train_pca)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_train, y_pred_prob_train),2))

In [ ]:
# create onfusion matrix
confusion = confusion_matrix(y_train, y_pred_train)

TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity/recall of our logistic regression model on train data
log_pca_train_recall = recall_score(y_train,y_pred_train )
log_pca_train_recall

In [ ]:
#accuracy on train data
log_pca_train_acc = round(metrics.accuracy_score(y_train, y_pred_train), 2)
log_pca_train_acc

#### Evaluation on test data

In [ ]:
# predict churn on test data
y_pred = model.predict(df_test_pca)

In [ ]:
# check area under curve
y_pred_prob = model.predict_proba(df_test_pca)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_test, y_pred_prob),2))

#### We observe 78% AUC on train data and 80% AUC on test data. Hence, this model is acceptable. We can move forward with calculating the sensitivity of the model

In [ ]:
# create onfusion matrix
confusion = confusion_matrix(y_test, y_pred)
confusion

In [ ]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# accuracy on test data
log_pca_test_acc = round(metrics.accuracy_score(y_test, y_pred), 2)
log_pca_test_acc

In [ ]:
#sensitivity/recall on test data
log_pca_test_recall = recall_score(y_test,y_pred )
log_pca_test_recall

In [ ]:
# Let us calculate specificity
specificity = TN / float(TN+FP)
print("Specificity: \t", round(specificity, 2))

In [ ]:
# f1 score
log_pca_test_f1 = f1_score(y_test, y_pred)
log_pca_test_f1

In [ ]:
#save the results.
log_pca_results = {
        result_column_names[0]: 'Logistic Regression with PCA', 
        result_column_names[1]: log_pca_train_acc,
        result_column_names[2]: log_pca_test_acc,
        result_column_names[3]: log_pca_train_recall,
        result_column_names[4]: log_pca_test_recall,
        result_column_names[5]: log_pca_test_f1,
}
model_results = model_results.append(log_pca_results, ignore_index=True)
model_results

#### With Logistic regression + PCA, we have attained a sensitivity of 81%. Let's now build a decision tree with PCA to see if we can do better

## Decision Tree Classifier with PCA and hyperparameter tuning

In [ ]:
dsTree = DecisionTreeClassifier(class_weight = 'balanced')
dsTreeParams = {
            'max_depth': [10, 20, 25, 28, 30, 35, 40],
            'min_samples_leaf': [50, 100, 200, 250, 300]}

dsFolds = KFold(n_splits = 5, random_state = 42)

dsTreeModel = GridSearchCV(estimator = dsTree, cv = dsFolds, param_grid = dsTreeParams, scoring='recall', 
                           verbose = 1, n_jobs = -1)

In [ ]:
# fit model
dsTreeModel.fit(df_train_pca, y_train)

In [ ]:
# print best hyperparameters
print("Best AUC: ", dsTreeModel.best_score_)
print("Best hyperparameters: ", dsTreeModel.best_params_)

#### Evaluation on train data

In [ ]:
dsTreeTrainPred = dsTreeModel.predict(df_train_pca)

In [ ]:
# create onfusion matrix
dsTreeTrainconfusion = confusion_matrix(y_train, dsTreeTrainPred)

In [ ]:
TP_ds = dsTreeTrainconfusion[1,1] # true positive 
TN_ds = dsTreeTrainconfusion[0,0] # true negatives
FP_ds = dsTreeTrainconfusion[0,1] # false positives
FN_ds = dsTreeTrainconfusion[1,0] # false negatives

# Let's see the sensitivity of our decision tree model
sensitivity_ds = TP_ds / float(TP_ds + FN_ds)
print("Sensitivity: \t", round(sensitivity_ds, 2))

In [ ]:
#sensitivity/recall on train data
dsTree_train_recall = recall_score(y_train,dsTreeTrainPred )
dsTree_train_recall

In [ ]:
# accuracy on train data
dsTree_train_acc = round(metrics.accuracy_score(y_train,dsTreeTrainPred), 2)
dsTree_train_acc

In [ ]:
# check area under curve
y_train_pred_prob_ds = dsTreeModel.predict_proba(df_train_pca)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_train, y_train_pred_prob_ds),2))

#### Evaluation on test data

In [ ]:
# predict churn on test data
dsTreePred = dsTreeModel.predict(df_test_pca)

In [ ]:
# create onfusion matrix
dsTreeconfusion = confusion_matrix(y_test, dsTreePred)
dsTreeconfusion

In [ ]:
TP_ds = dsTreeconfusion[1,1] # true positive 
TN_ds = dsTreeconfusion[0,0] # true negatives
FP_ds = dsTreeconfusion[0,1] # false positives
FN_ds = dsTreeconfusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our decision tree model
sensitivity_ds = TP_ds / float(TP_ds+FN_ds)
print("Sensitivity: \t", round(sensitivity_ds, 2))

In [ ]:
# Let us calculate specificity
specificity_ds = TN_ds / float(TN_ds+FP_ds)
print("Specificity: \t", round(specificity_ds, 2))

In [ ]:
#sensitivity/recall on train data
dsTree_test_recall = recall_score(y_test, dsTreePred)
dsTree_test_recall

In [ ]:
# accuracy on train data
dsTree_test_acc = round(metrics.accuracy_score(y_test, dsTreePred), 2)
dsTree_test_acc

In [ ]:
# check area under curve
y_pred_prob_ds = dsTreeModel.predict_proba(df_test_pca)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_test, y_pred_prob_ds),2))

In [ ]:
# f1 score
dsTree_test_f1 = f1_score(y_test, dsTreePred)
dsTree_test_f1

#### We observe 78% sensitivity on train data and 74% sensitivity on test data. Hence, we can accept this model. 

In [ ]:
#save the results.

# result_column_names = ["Model", "Train Accuracy", "Test Accuracy", "Train Recall", "Test Recall", "Test F1 Score"]
dsTree_results = {
        result_column_names[0]: 'Logistic Regression with PCA', 
        result_column_names[1]: dsTree_train_acc,
        result_column_names[2]: dsTree_test_acc,
        result_column_names[3]: dsTree_train_recall,
        result_column_names[4]: dsTree_test_recall,
        result_column_names[5]: dsTree_test_f1,
}
model_results = model_results.append(dsTree_results, ignore_index=True)
model_results

#### As observed, decision tree + PCA performs equally well on test data with 80% sesitivity. Let's build a random forest and try to improve the model

## Random Forest

In [ ]:
#random forest - the class weight is used to handle class imbalance - it adjusts the cost function
forest = RandomForestClassifier(class_weight='balanced', n_jobs = -1)

# hyperparameter space
params = { 
            'max_depth': [25, 28, 30, 35],
            'min_samples_leaf': [200, 300, 400],
            'n_estimators': [100, 150, 200]}

# create 5 folds
folds = KFold(n_splits = 5, shuffle = True, random_state = 42)

# create gridsearch object
rf_model = GridSearchCV(estimator=forest, cv=folds, param_grid=params, scoring='recall', n_jobs=-1, verbose=1)

In [ ]:
# fit model
rf_model.fit(X_train, y_train)

In [ ]:
# print best hyperparameters
print("Best AUC: ", rf_model.best_score_)
print("Best hyperparameters: ", rf_model.best_params_)

In [ ]:
# Creating the random forest from best parameters obtained above
rf_best_model = RandomForestClassifier(
    n_estimators = 200, 
    max_depth = 30,
    min_samples_leaf = 400,
    class_weight = 'balanced', 
    oob_score = True, 
    random_state = 4, 
    verbose = 1)

In [ ]:
# fit model
rf_best_model.fit(X_train, y_train)

#### Evaluation on train data

In [ ]:
# predict churn on train data
y_rf_train_pred = rf_best_model.predict(X_train)

In [ ]:
# create onfusion matrix
confusion = confusion_matrix(y_train, y_rf_train_pred)

TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

# Let's see the sensitivity of our random forest model
sensitivity = TP / float(TP+FN)
print("Sensitivity: \t", round(sensitivity, 2))

#### Evaluation on test data

In [ ]:
# predict churn on test data
y_rf_pred = rf_best_model.predict(X_test)

In [ ]:
# create onfusion matrix
confusion = confusion_matrix(y_test, y_rf_pred)
confusion

In [ ]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our random forest model
sensitivity = TP / float(TP+FN)
print("Sensitivity: \t", round(sensitivity, 2))

In [ ]:
# Let us calculate specificity
specificity = TN / float(TN+FP)
print("Specificity: \t", round(specificity, 2))

In [ ]:
# check area under curve
y_rf_pred_prob = rf_best_model.predict_proba(X_test)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_test, y_rf_pred_prob),2))

#### We observe 82% sensitivity on train data and 84% sensitivity on test data. Hence, we can accept this model.

*Note - We will be using this model later to derive the feature importance due to high interpretibility of Random Forests

## Random forest with PCA and hyperparameter tuning

In [ ]:
#random forest - the class weight is used to handle class imbalance - it adjusts the cost function
forest_pca = RandomForestClassifier(class_weight='balanced', n_jobs = -1)

# hyperparameter space
params_pca = {
            'max_depth': [25, 28, 30, 35, 40],
            'min_samples_leaf': [100, 200, 300, 400],
            'n_estimators': [50, 100, 150, 200]
}

# create 5 folds
folds_pca = KFold(n_splits = 5, shuffle = True, random_state = 4)

# create gridsearch object
rf_model_pca = GridSearchCV(estimator=forest, cv=folds, param_grid=params, scoring='recall', n_jobs=-1, verbose=1)

In [ ]:
# fit model
rf_model_pca.fit(df_train_pca, y_train)

In [ ]:
rf_model_pca.best_params_

#### Evaluation on train data

In [ ]:
y_pred_rf_train_pca = rf_model_pca.predict(df_train_pca)

In [ ]:
# create onfusion matrix
confusion_rf_pca = confusion_matrix(y_train, y_pred_rf_train_pca)

TP = confusion_rf_pca[1,1] # true positive 
TN = confusion_rf_pca[0,0] # true negatives
FP = confusion_rf_pca[0,1] # false positives
FN = confusion_rf_pca[1,0] # false negatives

# Let's see the sensitivity of our logistic regression model
sensitivity_rf_pca = TP / float(TP+FN)
print("Sensitivity: \t", round(sensitivity_rf_pca, 2))

#### Evaluation on test data

In [ ]:
y_pred_rf_pca = rf_model_pca.predict(df_test_pca)

In [ ]:
# create onfusion matrix
confusion_rf_pca = confusion_matrix(y_test, y_pred_rf_pca)
confusion_rf_pca 

In [ ]:
TP = confusion_rf_pca[1,1] # true positive 
TN = confusion_rf_pca[0,0] # true negatives
FP = confusion_rf_pca[0,1] # false positives
FN = confusion_rf_pca[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
sensitivity_rf_pca = TP / float(TP+FN)
print("Sensitivity: \t", round(sensitivity_rf_pca, 2))

In [ ]:
# Let us calculate specificity
specificity_rf_pca = TN / float(TN+FP)
print("Specificity: \t", round(specificity_rf_pca, 2))

In [ ]:
# check area under curve
y_pred_prob_rf_pca = rf_model_pca.predict_proba(df_test_pca)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_test, y_pred_prob_rf_pca),2))

#### We observe 77% sensitivity on train data and 79% sensitivity on test data. Hence, we can accept this model. But our previous model have performed slightly better than this model

## XGBoost with PCA

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# creating a KFold object 
folds_xg = 5

# specify range of hyperparameters
param_grid_xg = {
            'learning_rate': [0.1, 0.2, 0.3], 
             'subsample': [0.3, 0.4, 0.5],
             'max_depth': [2, 5],
             'scale_pos_weight': [10, 20, 30, 40]
            }          


# specify model
xgb_model = XGBClassifier(n_estimators=200, scale_pos_weight = 11.5)

# set up GridSearchCV()
model_cv_xg = GridSearchCV(estimator = xgb_model, 
                        param_grid = param_grid_xg, 
                        scoring= 'recall',
                        cv = folds_xg, 
                        n_jobs = -1,
                        verbose = 1,
                        return_train_score=True)     

In [ ]:
# fit the model
model_cv_xg.fit(df_train_pca, y_train)

In [ ]:
print('We can get accuracy of **'+str(round(model_cv_xg.best_score_,2))+'** using '+str(model_cv_xg.best_params_))

In [ ]:
model_cv_xg.best_params_

#### Evaluation on train data

In [ ]:
y_pred_xg_train = model_cv_xg.predict(df_train_pca)

In [ ]:
# create onfusion matrix
confusion_xg = confusion_matrix(y_train, y_pred_xg_train)

TP = confusion_xg[1,1] # true positive 
TN = confusion_xg[0,0] # true negatives
FP = confusion_xg[0,1] # false positives
FN = confusion_xg[1,0] # false negatives

# Let's see the sensitivity of our XGBoost model
sensitivity_xg = TP / float(TP+FN)
print("Sensitivity: \t", round(sensitivity_xg, 2))

#### Evaluation on test data

In [ ]:
y_pred_xg = model_cv_xg.predict(df_test_pca)

In [ ]:
# create onfusion matrix
confusion_xg = confusion_matrix(y_test, y_pred_xg)
confusion_xg 

In [ ]:
TP = confusion_xg[1,1] # true positive 
TN = confusion_xg[0,0] # true negatives
FP = confusion_xg[0,1] # false positives
FN = confusion_xg[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our XGBoost model
sensitivity_xg = TP / float(TP+FN)
print("Sensitivity: \t", round(sensitivity_xg, 2))

In [ ]:
# Let us calculate specificity
specificity_xg = TN / float(TN+FP)
print("Specificity: \t", round(specificity_xg, 2))

#### We observe 96% sensitivity on train data and 93% sensitivity on test data. Hence, we can accept this model

## Feature Importance

We can derive the feature importance from our Random Forest model which was built without PCA since the model already has feauture importance inbuilt

In [ ]:
# predictors
features = churn_filtered.drop('churn', axis=1).columns

# feature_importance
importance = rf_best_model.feature_importances_

# create dataframe
feature_importance = pd.DataFrame({'variables': features, 'importance_percentage': importance*100})
feature_importance = feature_importance[['variables', 'importance_percentage']]

# sort features
feature_importance = feature_importance.sort_values('importance_percentage', ascending=False).reset_index(drop=True)

In [ ]:
feature_importance.head(20)

## Interpretations & Conslusions 

- Average revenue per user seems to be most important feature in determining churn prediction
- Incoming and Outgoing Calls on romaing for 8th month are strong indicators of churn behaviour
- Roaming packs for both incoming and outgoing are strong predictors for churning
- Finally the data recharge for the 8th month is also an important feature for churn prediction

## Best model

In course of this assignment, we created several models:
- Logistic regression
- Logistic regression with PCA
- Decision tree with PCA
- Random forest
- Random forest with PCA
- XGBoost with PCA

In [ ]:
# models result summary 
model_results

Since, the business aims to predict the customers who will churn, hence, our model should have highest sensitivity as possible.
The best model observed is XGBoost with 93% sensitivity

------------------------------------------------------------------------------------------------------------------